## Install packages
GSpread to handle google sheets <br>
Pandas and Numpy for data handling and analysis <br>
Pyplot for visuals <br>
google.colab.auth for authentications <br>


In [1]:
#!pip install jupyter_datatables -q
#%pip install skimpy
import gspread_dataframe as gd
import gspread as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from jupyter_datatables import init_datatables_mode
# from google.colab import auth
# from google.auth import default
from skimpy import skim
from datetime import date
import time

In [ ]:
#init_datatables_mode()

## Authenticate with Google Sheets
use the authenticate_user() function from google.colab.auth to authenticate on your Google Sheets account and pass the credentials to GSpread using gs.authorize()

In [ ]:
auth.authenticate_user() # prompts google sign in
creds, _ = default() # get default credentials from auth

gc = gs.authorize(creds) # pass credentials to gspread

### Sheet URLS

In [2]:
VI = 'https://docs.google.com/spreadsheets/d/1bKwL32NDnkvZQdiba4pqfvvGDkqh9CBO3akwBV-uJEc/edit#gid=27' # Production Data, Input Sheet
LM = 'https://docs.google.com/spreadsheets/d/1vP8FzyoRfl1bnO4kSQa6t2xg5RdN4w8u0RAbd6rPzEM/edit#gid=0' # Shipping Data, Output Sheet for Cali Shipping Operations

In [3]:
# helper function for handling gsheets using gspread_dataframe
# has 3 modes, w - write: replace existing sheet data, a - append: keep existing data and add new
# r - read: read data from gsheets
# https://stackoverflow.com/questions/45540827/appending-pandas-data-frame-to-google-spreadsheet


def pandas_to_sheets(url,ws_name,df=None,mode='r'):

    # helper function for handling gsheets using gspread and gspread_dataframe
    # has 3 modes, w - write: replace existing sheet data, a - append: keep existing data and add new
    # r - read: read data from gsheets
    # https://stackoverflow.com/questions/45540827/appending-pandas-data-frame-to-google-spreadsheet


    ws = gc.open_by_url(url).worksheet(ws_name) # open worksheet by GSheet URL and worksheet name
    # clear and write new data to worsheet
    if(mode=='w'):
        ws.clear()
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=True,resize=True)
        return True
    # append new data to existing data in worksheet
    elif(mode=='a'):
        ws.add_rows(df.shape[0])
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=False,row=len(ws.get_all_values())+1,resize=False)
        return True
    # get data from worksheet as df, here including only necessary columns for shipping robot.
    else:
        return gd.get_as_dataframe(worksheet=ws,usecols=[2,3,4,5,9,15,16,20,22], header=1)

## Collect Data
Open VI spreadsheet and load data to pandas using created function

In [3]:
from functions import pandas_to_sheets
vi_df = pandas_to_sheets(VI,"PR") # get production data from gsheets using helper function
vi_df.fillna("",inplace=True) # Remove Nonetypes

In [ ]:
# this uses gspread vanilla and creates the dataframe from a selection of rows
# code within pandas_to_sheets accomplishes this in a single line


# vi_sh = gc.open_by_url(VI) # Open Workbook
# vi_sh = vi_sh.worksheet('PR') # Select Worksheet
# vi_df = pd.DataFrame(vi_sh.get('B2:W900')) # Select Range for Import or use get_all_records()
# vi_df.columns = vi_df.iloc[0] # Set Header Row
# vi_df = vi_df[1:].copy() # Drop Duplicate Row
# vi_df.fillna("",inplace=True) # Remove Nonetypes

In [5]:
# skim(vi_df)

IndexError: list index out of range

Drop unecessary columns

In [ ]:
# deprecated by pandas_to_sheets

# vi_df.rename(columns={'® = REVISED' : 'Build'}, inplace=True) # rename first column
# c = ['VIN #', 'Body Type', 'Truck Type', 'Customer','Notes','Est. Ship Date', \
#      'Delivery Cost', 'Shipper', 'Shipping Arranger', \
#      'Pick up #', 'Shipping Address' ] # Select Columns to Keep
# inv_df = vi_df[c].copy() # Copy Columns to Keep to new DF
# inv_df.replace(r'^\s*$', np.NaN, regex=True, inplace=True) # Fill Empty Cells with NaN

In [9]:
vi_df.iloc[55:70]

,VIN #,Body Type,Truck Type,Customer,Notes,Est. Ship Date,Delivery Cost,Shipping Arranger,Shipping Address
55,,,,,,,,,
56,,,,,,,,,
57,352800,2000 SS TANK - NFPA,KW380ABPX7AT33124,"DEREK ZAGARELLA - MURRIETTA, CA",2/22 DEPOSIT RECVD,,0,WILL CALL,"23748 CLOVERLEAF WAY, MURRIETTA, CA 92562"
58,TBD,2000 TANK,F750HBPOWAT26084,"ARNOLD MACHINERY - N. LAS VEGAS, NV","3/2 purchase docs sent for signatures, missing...",,0,WILL CALL,"4136 DONOVAN WAY, N. LAS VEGAS, NV 89030"
59,,,,,,,,,
60,,,,,vsdgg,,,,
61,,,,,,,,,
62,1FDNF7DC0PDF02247,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - STOCKTON, CA",3/7 rec'd check (hold 3-5 days) 3/6 shippin...,,1900,TBD,"2500 NAVY DR, STOCKTON, CA 95206"
63,1FDPF7DC5PDF02169,2000 TANK *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - SANTEE, CA","3/6 updated invoice sent to customer, missing ...",,1000,VALEW,"12001 FANITA PARKWAY, SANTEE CA 92071"
64,1FDNF7DC2PDF02251,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"ARI / UNITED - HAMPTON, VA","UNITED PO 18082049, 3/6 shipping docs ready",03/08/23,6200,VALEW,"2327 W PEMBROKE, AVE HAMPTON, VA 23661"


Select Cali and TX Completed Units and Seperate into Individual DFs

In [4]:
ca_start = vi_df[vi_df['Shipping Arranger'].str.match('V_START', na=False)].index[0] # find start of CA Completed units
cdf = vi_df.iloc[ca_start+2:] # remove data before selection
ca_end = cdf[cdf['VIN #'] == ''].index[0] # find end of CA Completed units
ca_df = vi_df.iloc[ca_start+2:ca_end].copy() # Select CA Completed Units into new DF

In [7]:
# skim(ca_df)

IndexError: list index out of range

In [5]:
tx_start = vi_df[vi_df['VIN #'].str.match('COMPLETED EAST', na=False)].index[0] # find start of TX Completed units
tdf = vi_df.iloc[tx_start+1:] # remove data before selection
tx_end = tdf[tdf['VIN #'] == ''].index[0] # find end of TX Completed units
tx_df = vi_df.iloc[tx_start+1:tx_end].copy() # Select TX Completed Units into new DF


In [7]:
tx_df.tail()

,VIN #,Body Type,Truck Type,Customer,Notes,Est. Ship Date,Delivery Cost,Shipping Arranger,Shipping Address
126,1003527,TK-4000-AC,TANK KIT,"MARTIN MARIETA - SANTA MARIA, CA",3/7 final invoice freight sent,,3475,1954 MFG,"1341 WEST MCCOY LN, SANTA MARIA, CA 93455"
127,3ALHCYFE7PDNW2690,4000 TANK,M2106ABISLAT58132,"4 RIVERS - FARMINGTON, NM","3/7 paid in full, ready to be picked up",,,WILL CALL,
128,1FDNF7DC9PDF12940,10' DW DUMP,F750HBPOWAT26084,"VANDALIA RENTAL - VANDALIA, OH","3/7 rec'd signed docs, missing funds DOCS SE...",,0,WILL CALL,"11063 ENGLE ROAD, VANDALIA, OH 45377"
129,1FDNF7DC0PDF12941,10' DW DUMP,F750HBPOWAT26084,"VANDALIA RENTAL - VANDALIA, OH","3/7 rec'd signed docs, missing funds DOCS SE...",,0,WILL CALL,"11063 ENGLE ROAD, VANDALIA, OH 45377"
130,WT12K23023,WATER TOWER,WATER TOWER,"UNITED RENTALS - HENDERSON, CO","2/2 signed docs rcvd, ready to ship 3/9",,1915,1954 MFG,"9700 ALTON WAY, HENDERSON, CO. 80640"


In [8]:
ca_df['Origin'] = ["CA" for x in range(len(ca_df))]
tx_df['Origin'] = ["TX" for x in range(len(tx_df))]

In [9]:
joined_df = ca_df.append(tx_df)
joined_df.head()

,VIN #,Body Type,Truck Type,Customer,Notes,Est. Ship Date,Delivery Cost,Shipping Arranger,Shipping Address,Origin
66,1FDNF7DC0PDF02247,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - STOCKTON, CA",3/7 rec'd check (hold 3-5 days) 3/6 shippin...,03/13/23,1900,VALEW,"2500 NAVY DR, STOCKTON, CA 95206",CA
67,1FDPF7DC5PDF02169,2000 TANK *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - SANTEE, CA","3/8 rec'd originals, missing funds",,1000,VALEW,"12001 FANITA PARKWAY, SANTEE CA 92071",CA
68,1FDNF7DC2PDF02251,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"ARI / UNITED - HAMPTON, VA","UNITED PO 18082049, 3/6 shipping docs ready",03/09/23,6200,VALEW,"2327 W PEMBROKE, AVE HAMPTON, VA 23661",CA
69,3ALHCYFE5PDNV6628,4000 TANK,M2106ABISLAT58132,"CASHMAN EQUIPMENT - HENDERSON, NV",3/8 shipping docs ready,03/10/23,1600,VALEW,"3300 SAINT ROSE PARKWAY, HENDERSON NV 89052",CA
70,1FDNF7DCXPDF02224,10' DUMP,F750HBPOWAT26084,"ARI / UNITED - ALEXANDRA, LA","UNITED PO 18078843, 3/8 shipping docs ready",03/09/23,4600,VALEW,"846 DALLAS AVENUE ALEXANDRIA, LA 71302",CA


Filter out Will Call and other units not ready for shipping

In [10]:
# Filter DFs to only include ready to ship units that have not been scheduled
ca_df['Notes'] = ca_df['Notes'].str.lower() # lowercase for string matching
ca_df = ca_df[(ca_df['Notes'].str.contains('shipping docs ready|ready to ship')) & (ca_df['Shipping Arranger'] != 'WILL CALL') & (ca_df['Est. Ship Date'] == "")] # filter DF

tx_df['Notes'] = tx_df['Notes'].str.lower() # lowercase for string matching
tx_df = tx_df[(tx_df['Notes'].str.contains('shipping docs ready|ready to ship')) & (tx_df['Shipping Arranger'] != 'WILL CALL') & (tx_df['Est. Ship Date'] == "")] # filter DF

ca_qty = len(ca_df)
tx_qty = len(tx_df)

In [11]:
# remove duplicate location info from customer column
ca_df['Customer'] = ca_df["Customer"].str.extract(r'(.*-)')
tx_df['Customer'] = tx_df["Customer"].str.extract(r'(.*-)')

# Create Shipping Tables

In [12]:
# reference dictionaries

# dictionary of Dimensions
dims = {
    "4000" : "35'L x 8'W x 10'H x 22,000lbs",
    "2000" : "25'L x 8'W x 9'H x 15,000lbs",
    "10" : "25'L x 8'W x 9'H x 15,000lbs",
    "15" : "30'L x 8'W x 9'H x 15,000lbs",
    "WATER TOWER" : "36'L x 8'W x 13'H x 19,500lbs"
}
# dictionary of Models
models = {
    "MA" : "MACK",
    "M2" : "FRLR",
    "SD" : "FRLR",
    "F7" : "FORD",
    "F5" : "FORD",
    "F6" : "FORD",
    "KW" : "KW",
    "PB" : "PB",
    "INT" : "INT"
}
# dictionary of Axle quantities
axles = {
    "132" : "3-Axle",
    "084" : "2-Axle",
    "096" : "2-Axle",
    "108" : "2-Axle",
    "120" : "2-Axle",
    "168" : "2-Axle",
    "170" : "2-Axle" 
}

origins = {
    "TX" : "1954 MFG - 4688 SH-16, GRAHAM, TX 76450",
    "CA" : "VALEW - 12522 VIOLET RD, ADELANTO, CA 92301"
}

In [14]:
joined_df.head()

,VIN #,Body Type,Truck Type,Customer,Notes,Est. Ship Date,Delivery Cost,Shipping Arranger,Shipping Address,Origin
66,1FDNF7DC0PDF02247,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - STOCKTON, CA",3/7 rec'd check (hold 3-5 days) 3/6 shippin...,03/13/23,1900,VALEW,"2500 NAVY DR, STOCKTON, CA 95206",CA
67,1FDPF7DC5PDF02169,2000 TANK *CUSTOM PAINT*,F750HBPOWAT26084,"W.M LYLES - SANTEE, CA","3/8 rec'd originals, missing funds",,1000,VALEW,"12001 FANITA PARKWAY, SANTEE CA 92071",CA
68,1FDNF7DC2PDF02251,10' DW DUMP *CUSTOM PAINT*,F750HBPOWAT26084,"ARI / UNITED - HAMPTON, VA","UNITED PO 18082049, 3/6 shipping docs ready",03/09/23,6200,VALEW,"2327 W PEMBROKE, AVE HAMPTON, VA 23661",CA
69,3ALHCYFE5PDNV6628,4000 TANK,M2106ABISLAT58132,"CASHMAN EQUIPMENT - HENDERSON, NV",3/8 shipping docs ready,03/10/23,1600,VALEW,"3300 SAINT ROSE PARKWAY, HENDERSON NV 89052",CA
70,1FDNF7DCXPDF02224,10' DUMP,F750HBPOWAT26084,"ARI / UNITED - ALEXANDRA, LA","UNITED PO 18078843, 3/8 shipping docs ready",03/09/23,4600,VALEW,"846 DALLAS AVENUE ALEXANDRIA, LA 71302",CA


In [13]:
# create item descriptions
import re
a1 = time.time()
test = []

for row in ca_df.itertuples(index=False, name=None):
  vin = row[0][-6:]
  model = models[row[2][:2]]
  axle = axles[row[2][-3:]]
  item = f"{model} {axle} TRUCK X 1"
  origin = origins['CA']
  dest = f"{row[3]} {row[-1]}"
  rate = row[6]
  dim = dims[re.search(r'(^\d+)|(WATER TOWER)', row[1])[0]]
  rts = date.today().strftime("%m/%d/%Y") 

  test.append({"STATUS":"", "LOAD #":"", "VIN" : vin, "TOTAL DIMENSIONS" : dim, "ORIGIN": origin, "DESTINATION" : dest,
           "ITEM DESCRIPTION" : item, "RATE" : rate, "READY TO SHIP" : rts})
test_df = pd.DataFrame(test)
b1 = time.time()
print(b1-a1)
test_df.head()

0.0010001659393310547


""


In [18]:
# test perfomance of list comprehension to collect data and iterows. What is itertuples? Didnt need itertuples, solved with list comprehension,
# still would like to time the operations of each for comparison
a = time.time()
# list comprehension
ca_vins = [x[-6:] for x in ca_df['VIN #']] # extract last 6 digits of VIN #
ca_dims = [dims[x] for x in ca_df['Body Type'].str.extract(r'(^\d+)|(WATER TOWER)')[0]] # lookup dimensions for body style in created dict
ca_dest = [x for x in ca_df[['Customer', 'Shipping Address']].apply(' '.join, axis=1)] # Comine Customer Name and Shipping Address for Destination Info
ca_item = [f'{models[x[:2]]} {axles[x[-3:]]} Truck X 1' for x in ca_df['Truck Type']] # create item descriptions from lookups
ca_rate = [x for x in ca_df['Delivery Cost']] # get delivery cost
ca_rts = [date.today().strftime("%m/%d/%Y") for x in range(len(ca_df))] # todays date
ca_origin = [origins["CA"] for x in range(len(ca_df))] # origin address
ca_stat = ['TBD' for x in range(len(ca_df))]
ca_id = ['TBD' for x in range(len(ca_df))]

ca_dict = {"STATUS":"", "LOAD #":"", "VIN" : ca_vins, "TOTAL DIMENSIONS" : ca_dims, "ORIGIN": ca_origin, "DESTINATION" : ca_dest,
           "ITEM DESCRIPTION" : ca_item, "RATE" : ca_rate, "READY TO SHIP" : ca_rts}

ca_shipping = pd.DataFrame(ca_dict)
b = time.time()
print(b-a)
ca_shipping.head(6)

0.00793600082397461


,STATUS,LOAD #,VIN,TOTAL DIMENSIONS,ORIGIN,DESTINATION,ITEM DESCRIPTION,RATE,READY TO SHIP


Iterate through TX Completed DF, Create TX Shipping Info Table

In [ ]:
# last row for appending df
ws = gc.open_by_url(CA_LM).worksheet("TX")
len(ws.get_all_values())+1

115

Write new shipping info to Output Sheets. Avoid Duplicates.

In [ ]:
# https://stackoverflow.com/questions/5458048/how-can-i-make-a-python-script-standalone-executable-to-run-without-any-dependen
# https://medium.com/@alexandre.tkint/send-pandas-data-frame-via-email-in-python-c346314321fa

In [ ]:
q_df = pandas_to_sheets(VI,"PR") # get production data from gsheets using helper function
q_df.fillna("",inplace=True) # Remove Nonetypes

In [ ]:
q_df.isna().sum()

VIN #                0
Body Type            0
Truck Type           0
Customer             0
Notes                0
Est. Ship Date       0
Delivery Cost        0
Shipping Arranger    0
Shipping Address     0
dtype: int64

In [ ]:
# select rows where Delivery Cost is empty, Shipping arranger is Valew or 1954, Customer contains United, and Vin number is present

q_df['Customer'] = q_df['Customer'].str.lower() # lowercase all letters for string matching
q_df = q_df[(q_df['Customer'].str.contains('united')) & (q_df['Shipping Arranger'] != 'WILL CALL') & (q_df['Delivery Cost'].str.match("|TBD")) & (q_df['VIN #'] != 'TBD')] # filter DF


In [ ]:
q_df = pd.DataFrame(q_df.groupby(['Customer','Body Type']).count()['VIN #'])
q_df.head(30)

VIN #
Customer                                  Body Type         
ari / united - tbd                        10' DUMP         3
                                          2000 TANK        2
united rentals - henderson, co            WATER TOWER      6
united rentals - south salt lake city, ut WATER TOWER      3
united rentals hold conexpo               WATER TOWER      1

Emailing a DF

wrap this up in a function and document it please

https://mailtrap.io/blog/python-send-email-gmail/

https://stackoverflow.com/questions/50564407/pandas-send-email-containing-dataframe-as-a-visual-table

https://stackoverflow.com/questions/30314538/attaching-file-in-python-email

In [ ]:
import os
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
# from email.mime.base import MIMEBase
# from email import encoders
import smtplib
from io import StringIO
# def gmail_df(sender, recipients, subject, df, password):

sender = '1954jacobf@gmail.com'
recipients = 'jacob@1954mfg.com'

if type(recipients) == list:
  emaillist = [elem.strip().split(',') for elem in recipients]
else:
  emaillist = recipients

msg = MIMEMultipart()
msg['Subject'] = "To Quote"
msg['From'] = sender


html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(q_df.to_html())

msg_txt = MIMEText(html, 'html')
msg.attach(msg_txt)

# q_df.to_csv("attachment.csv")

# with open("attachment.csv", "rb") as attachment:
#       # Add the attachment to the message
#       msg_attach = MIMEBase("application", "octet-stream")
#       msg_attach.set_payload((attachment).read())
# encoders.encode_base64(msg_attach)
# msg_attach.add_header(
# "Content-Disposition",
# f"attachment; filename= attachment.csv")
# msg.attach(msg_attach)

textStream = StringIO()
q_df.to_csv(textStream,index=True)
msg.attach(MIMEApplication(textStream.getvalue(), Name="to_quote.csv"))


password = "tqahcfjmgckkgcpc"

# SMTP.starttls() #[keyfile[, certfile]]
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(sender, password)
server.sendmail(msg['From'], emaillist , msg.as_string())
server.quit()

(221,
 b'2.0.0 closing connection f24-20020ac840d8000000b003b7e8c04d2esm8059607qtm.64 - gsmtp')